1. Download and load the data. Perform the initial exploration and cleaning.
   - it is already split into training and testing parts
   - labels go from 0 to 76
   - checks should include e.g.:
     - checking data types, number of texts,
     - plotting the class distribution
     - plotting the histogram of text lengths


In [ ]:
from datasets import load_dataset

dataset = load_dataset("PolyAI/banking77")

train_df = pl.DataFrame(dataset["train"])
test_df = pl.DataFrame(dataset["test"])

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.columns

In [ ]:
class_counts = train_df.groupby("label").agg(pl.count()).to_pandas()

plt.figure(figsize=(12, 6))
plt.bar(class_counts['label'], class_counts['count'], color='skyblue')
plt.xlabel('Class Labels')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Banking77 Dataset')
plt.xticks(class_counts['label'])

In [ ]:
text_lengths = train_df['label'].str.len()